# CycleGAN

**Objective:** Implement CycleGAN for horses2zebra dataset

**Paper**:
[Unpaired Image to Image Translation using Cycle-Consistent GAN](https://arxiv.org/pdf/1703.10593.pdf)

Some important points to note about the CycleGAN:
* The goal is to learn mapping functions between two domains X & Y.
*  Model includes the following things:
  * Mapping G: X -> Y
  * Mapping F : Y -> X
  * Two Adversarial Discriminators:
    * $D_x$ to distinguish between images {x} and generated image {F{y}}
    * $D_y$ to distinguish between y and generated image {G{x}}
    
* Objective contains the following losses:
  * **Adversarial loss:** matching the distribution of the generated images to the data distribution of the target domain.
  
  * **Cycle-consistency loss**: to prevent the mappings   G & F from contradicting each other.
  


## Import Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!cp '/content/drive/My Drive/App/CycleGAN/horse2zebra.zip' /content

In [3]:
!unzip -q /content/horse2zebra.zip

replace horse2zebra/trainA/n02381460_6223.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [0]:
#!cp '/content/drive/My Drive/App/CycleGAN/trainA.64.zip' /content
#!cp '/content/drive/My Drive/App/CycleGAN/trainB.64.zip' /content

In [0]:
#!unzip -q /content/trainA.64.zip /content
#!unzip -q /content/trainB.64.zip /content

In [0]:
#!mv /content/content/trainA.64.npy /content
#!mv /content/content/trainB.64.npy /content

## Import Necessary Packages

In [7]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-89sov9h0
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-89sov9h0
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=30295ea1a7a071c004cf95bf71d2b598863a39db28f65af8bb1c3e1fa7814d13
  Stored in directory: /tmp/pip-ephem-wheel-cache-pqweu7i5/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [8]:
!pip install pillow

## Imports

In [0]:
import numpy as np
import os
from PIL import Image
import scipy
#import scipy.misc
#import imageio
import cv2
from glob import glob
import datetime

import matplotlib.pyplot as plt
%matplotlib inline

## Configurations

In [0]:
#TRAIN_A_DATA_FILE = '/content/trainA.64.npy'
#TRAIN_B_DATA_FILE = '/content/trainB.64.npy'

DATA_LOC = '/content/horse2zebra'
TRAIN_A_LOC = os.path.join(DATA_LOC, 'trainA') 
TRAIN_B_LOC = os.path.join(DATA_LOC, 'trainB') 
TEST_A_LOC  = os.path.join(DATA_LOC, 'testA') 
TEST_B_LOC  = os.path.join(DATA_LOC, 'testB') 

# image details
img_rows = 64
img_cols = 64
channels = 3
image_shape = (img_rows, img_cols, channels)
img_res = 64

## Latent Space (vectors) dimension
latent_dim = 100

# Discriminator - PatchGAN, the filters doubles at every stage
# 64, 128, 256 and 512
dis_num_start_filters = 64

# calculate the output shape of Discriminator (PatchGAN)
patch_size = int(img_rows / 2**4)
disc_patch = (patch_size, patch_size, 1)

# Generator
gen_num_start_filters = 32

# Loss weights
lambda_cycle = 10.0                    # Cycle-consistency loss
lambda_ident = 0.1 * lambda_cycle      # Identity loss


## Get the Data and Normalize

In [0]:
?cv2.imread

In [0]:
def image_read(path):
    #return scipy.misc.imread(path, mode='RGB').astype(np.float)
    #return imageio.imread(path).astype(np.float)
    # this will be bgr image.
    return cv2.imread(path).astype(float)

In [0]:
# load images 
# output: batch of images
def load_images(domain, batch_size=1, is_testing=False):
  # trainA, trainB, testA, testB
  data_type = "train%s" % domain if not is_testing else "test%s" % domain
  
  path = glob('%s/%s/*' % (DATA_LOC, data_type))

  batch_images = np.random.choice(path, size=batch_size)

  imgs = []
  for img_path in batch_images:
      img = image_read(img_path)
      if not is_testing:
          img = cv2.resize(img, (img_res,img_res))

          if np.random.random() > 0.5:
              img = np.fliplr(img)
      else:
          img = cv2.resize(img, (img_res, img_res))
      imgs.append(img)

  imgs = np.array(imgs)/127.5 - 1.

  return imgs

In [0]:
# like generator, keep supply the batches of images
def generate_batch_images(batch_size=1, is_testing=False):
  # get the domain
  data_type = "train" if not is_testing else "test"
  
  path_A = glob('%s/%sA/*' % (DATA_LOC, data_type))
  path_B = glob('%s/%sB/*' % (DATA_LOC, data_type))

  n_batches = int(min(len(path_A), len(path_B)) / batch_size)
  total_samples = n_batches * batch_size

  # Sample n_batches * batch_size from each path list so that model sees all
  # samples from both domains
  path_A = np.random.choice(path_A, total_samples, replace=False)
  path_B = np.random.choice(path_B, total_samples, replace=False)

  for i in range(n_batches-1):
      batch_A = path_A[i*batch_size:(i+1)*batch_size]
      batch_B = path_B[i*batch_size:(i+1)*batch_size]
      imgs_A, imgs_B = [], []
      for img_A, img_B in zip(batch_A, batch_B):
          img_A = image_read(img_A)
          img_B = image_read(img_B)

          img_A = cv2.resize(img_A, (img_res,img_res))
          img_B = cv2.resize(img_B, (img_res,img_res))

          if not is_testing and np.random.random() > 0.5:
                  img_A = np.fliplr(img_A)
                  img_B = np.fliplr(img_B)

          imgs_A.append(img_A)
          imgs_B.append(img_B)

      imgs_A = np.array(imgs_A)/127.5 - 1.
      imgs_B = np.array(imgs_B)/127.5 - 1.

      yield imgs_A, imgs_B 

## Build CycleGAN

In [14]:
# keras layers
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D

# from keras_contrib
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

from keras.models import Sequential, Model

from keras.optimizers import Adam

Using TensorFlow backend.


## Discriminator

For Discriminator:
*  Use PatchGAN - only penalizes the structure at the scale of patches.
* PatchGAN classifies the NxN patch is real or fake
*  They have fewer parameters than the full image discriminator
* PatchGAN are used in [Image to Image translation](https://arxiv.org/pdf/1611.07004.pdf)

In [0]:
# Discriminator layer has the following
#  * Conv2D - filter size: 4x4, strides:2
#  * LeakyReLU
#  * InstanceNormalization
#
def d_layer(layer_input, filters, f_size=4, normalization=True):
  d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
  d = LeakyReLU(alpha=0.2)(d)
  if normalization:
      d = InstanceNormalization()(d)
  return d

In [0]:
# build discriminator uses PatchGAN
# Uses the patch to classify the image is fake or real.
# PatchGAN uses 
#  * kernel size 4x4
#  * num filters double at each stage
def build_discriminator():
  img = Input(image_shape)
  
  d1 = d_layer(img, dis_num_start_filters, normalization=False)
  d2 = d_layer(d1, dis_num_start_filters*2)
  d3 = d_layer(d2, dis_num_start_filters*4)
  d4 = d_layer(d3, dis_num_start_filters*8)

  validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

  return Model(img, validity)

## Generator

Generator can be one of the following two things:

     * Encoder : Decoder combo
     or
     * Encoder : Transformer : Decoder
     
 The Encoder shrinks the input image. Uses Conv layers (with strides:2).
 
 The Transformer uses residual blocks
 
 The Decoder expands the image with transpose Conv.
 
 Note: each layer will use LeakyReLU and InstanceNormalization

In [0]:
# define encoder layer

def encoder_layer(layer_input, filters, f_size=4):
  # Downsamples the input 
  d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
  d = LeakyReLU(alpha=0.2)(d)
  d = InstanceNormalization()(d)
  return d

# define decoder layer
def decoder_layer(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
  # upsample the input
  u = UpSampling2D(size=2)(layer_input)
  u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
  if dropout_rate:
      u = Dropout(dropout_rate)(u)
  u = InstanceNormalization()(u)
  u = Concatenate()([u, skip_input])
  return u

In [0]:
# define the generator (image -> image)
#  
def build_generator():
  # Image input
  d0 = Input(shape=image_shape)

  # Downsample
  d1 = encoder_layer(d0, gen_num_start_filters)
  d2 = encoder_layer(d1, gen_num_start_filters*2)
  d3 = encoder_layer(d2, gen_num_start_filters*4)
  d4 = encoder_layer(d3, gen_num_start_filters*8)

  # Upsample
  u1 = decoder_layer(d4, d3, gen_num_start_filters*4)
  u2 = decoder_layer(u1, d2, gen_num_start_filters*2)
  u3 = decoder_layer(u2, d1, gen_num_start_filters)

  u4 = UpSampling2D(size=2)(u3)
  output_img = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

  return Model(d0, output_img)

## Build CycleGAN

In [0]:
optimizer = Adam(0.0002, 0.5)

###  Discriminators

In [20]:
# build and compile discriminators
#
dis_A = build_discriminator()
dis_B = build_discriminator()

dis_A.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
dis_B.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

W0804 10:06:17.310028 140151751292800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0804 10:06:17.313156 140151751292800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0804 10:06:17.319377 140151751292800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0804 10:06:17.633653 140151751292800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



###   Generators

In [21]:
# build and compile generators
gen_AB = build_generator()
gen_BA = build_generator()

# input images for both domains
image_A = Input(shape=image_shape)

# translate the image to other domain
fake_B  = gen_AB(image_A)

# reconstructed image
reconstr_A = gen_BA(fake_B)

# other image as well.
image_B = Input(shape=image_shape)
fake_A  = gen_BA(image_B)
reconstr_B = gen_AB(fake_A)

# identify mappings for both images
ident_A = gen_BA(image_A)
ident_B = gen_AB(image_B)

# for the combined model, train only generators.
# so, freeze the discriminators
dis_A.trainable = False
dis_B.trainable = False

# discriminators detemine the authenticity of the translated images
result_A = dis_A(fake_A)
result_B = dis_B(fake_B)

# combined model
combined = Model(inputs = [image_A, image_B],
                 outputs = [result_A, result_B, 
                            reconstr_A, reconstr_B, ident_A, ident_B])

# compile
combined.compile(loss=['mse', 'mse', 
                       'mae', 'mae',
                       'mae', 'mae'],
                 loss_weights=[  1, 1,
                                 lambda_cycle, lambda_cycle,
                                 lambda_ident, lambda_ident ],
                  optimizer=optimizer)

W0804 10:06:17.814677 140151751292800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.



## Train

In [0]:
np.random.seed(123)

In [0]:
# sample images from test suite to see the progress.
def sample_images(epoch, batch_idx):
  os.makedirs('/content/gen_images', exist_ok=True)
  r, c = 2, 3

  imgs_A = load_images(domain="A", batch_size=1, is_testing=True)
  imgs_B = load_images(domain="B", batch_size=1, is_testing=True)

  # Demo (for GIF)
  #imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
  #imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

  # Translate images to the other domain
  fake_B = gen_AB.predict(imgs_A)
  fake_A = gen_BA.predict(imgs_B)
  
  # Translate back to original domain
  reconstr_A = gen_BA.predict(fake_B)
  reconstr_B = gen_AB.predict(fake_A)

  gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, 
                             imgs_B, fake_A, reconstr_B])

  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5

  titles = ['Original', 'Translated', 'Reconstructed']
  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
      for j in range(c):
          axs[i,j].imshow(gen_imgs[cnt])
          axs[i, j].set_title(titles[j])
          axs[i,j].axis('off')
          cnt += 1
  fig.savefig("/content/gen_images/%d_%d.png" % (epoch, batch_idx))
  plt.close()

In [0]:
#!rm -rf /content/gen_images

In [0]:
epochs = 50
batch_size = 1
sample_interval = 500

In [38]:
# Adversarial loss ground truths
valid = np.ones((batch_size,) + disc_patch)
fake = np.zeros((batch_size,) + disc_patch)

start_time = datetime.datetime.now()

for epoch in range(epochs):
  
  
  #process in batches
  for batch_i, (imgs_A, imgs_B) in enumerate(generate_batch_images(batch_size)):
  
    # ----------------------
    #  Train Discriminators
    # ----------------------

    # Translate images to opposite domain
    fake_B = gen_AB.predict(imgs_A)
    fake_A = gen_BA.predict(imgs_B)

    # Train the discriminators (original images = real / translated = Fake)
    dA_loss_real = dis_A.train_on_batch(imgs_A, valid)
    dA_loss_fake = dis_A.train_on_batch(fake_A, fake)
    dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

    dB_loss_real = dis_B.train_on_batch(imgs_B, valid)
    dB_loss_fake = dis_B.train_on_batch(fake_B, fake)
    dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

    # Total disciminator loss
    d_loss = 0.5 * np.add(dA_loss, dB_loss)

    # ------------------
    #  Train Generators
    # ------------------

    # Train the generators
    g_loss = combined.train_on_batch([imgs_A, imgs_B],
                                     [valid, valid, 
                                      imgs_A, imgs_B,
                                      imgs_A, imgs_B]) 
    
    elapsed_time = datetime.datetime.now() - start_time
    
    if batch_i % sample_interval == 0:
      print ("[Epoch %d/%d] [Batch %d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                        % ( epoch, epochs,
                                                                            batch_i,  
                                                                            d_loss[0], 100*d_loss[1],
                                                                            g_loss[0],
                                                                            np.mean(g_loss[1:3]),
                                                                            np.mean(g_loss[3:5]),
                                                                            np.mean(g_loss[5:6]),
                                                                            elapsed_time))
      sample_images(epoch, batch_i)
   

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/50] [Batch 0] [D loss: 0.344585, acc:  45%] [G loss: 9.771956, adv: 0.540866, recon: 0.389615, id: 0.449109] time: 0:00:00.164245 
[Epoch 0/50] [Batch 500] [D loss: 0.273496, acc:  43%] [G loss: 5.784718, adv: 0.477119, recon: 0.212756, id: 0.302350] time: 0:01:02.058005 
[Epoch 0/50] [Batch 1000] [D loss: 0.251443, acc:  54%] [G loss: 4.000812, adv: 0.494759, recon: 0.129137, id: 0.183176] time: 0:02:03.212105 
[Epoch 1/50] [Batch 0] [D loss: 0.185995, acc:  64%] [G loss: 7.728089, adv: 0.557735, recon: 0.297731, id: 0.281763] time: 0:02:11.517422 
[Epoch 1/50] [Batch 500] [D loss: 0.427785, acc:  26%] [G loss: 5.238328, adv: 0.711084, recon: 0.170138, id: 0.241952] time: 0:03:13.073731 
[Epoch 1/50] [Batch 1000] [D loss: 0.169545, acc:  75%] [G loss: 4.297024, adv: 0.721930, recon: 0.125548, id: 0.142691] time: 0:04:14.319814 
[Epoch 2/50] [Batch 0] [D loss: 0.141728, acc:  81%] [G loss: 4.470566, adv: 0.687624, recon: 0.136154, id: 0.144231] time: 0:04:22.445932 
[Epoch 2/5

In [0]:
!cp /content/gen_images/47_500.png '/content/drive/My Drive/App/CycleGAN/gen_images' 
!cp /content/gen_images/48_500.png '/content/drive/My Drive/App/CycleGAN/gen_images' 
!cp /content/gen_images/49_500.png '/content/drive/My Drive/App/CycleGAN/gen_images' 